In [69]:
import pandas as pd
import astropy
from astropy import units as u
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation, GeocentricTrueEcliptic
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
import numpy as np

In [100]:
t = Time('2024-04-13T21:08:00')
m1_2024 = pd.read_csv('./Stellarium_ephemeris/2024_m1_yv.csv')

m1_2024_coords = [SkyCoord(ra=Angle(m1_2024[' RA (J2000)'][i]), dec=Angle(m1_2024[' Dec (J2000)'][i]), frame='icrs') for i in range(len(m1_2024[' RA (J2000)']))]
m1_2024_longs = [m1_2024_coords[i].transform_to(GeocentricTrueEcliptic(equinox=t+i*8)).lon.degree for i in range(5)]

print(m1_2024_longs)
for i in range(4):
    print(Angle(m1_2024_longs[i+1]-m1_2024_longs[i]-8, unit='deg'))

[24.197027968635425, 32.01838627612801, 39.804656887963574, 47.56286379685391, 55.293102227456615]
-0d10m43.11009303s
-0d12m49.42579739s
-0d14m30.45512799s
-0d16m11.14164983s


In [101]:
t = Time('2023-04-14T15:05:00')
m1_2023 = pd.read_csv('./Stellarium_ephemeris/2023_m1_yv.csv')

m1_2023_coords = [SkyCoord(ra=Angle(m1_2023[' RA (J2000)'][i]), dec=Angle(m1_2023[' Dec (J2000)'][i]), frame='icrs') for i in range(len(m1_2023[' RA (J2000)']))]
m1_2023_longs = [m1_2023_coords[i].transform_to(GeocentricTrueEcliptic(equinox=t+i*8)).lon.degree for i in range(5)]

print(m1_2023_longs)
for i in range(4):
    print(Angle(m1_2023_longs[i+1]-m1_2023_longs[i]-8, unit='deg'))

[24.183559401274398, 32.008898289443316, 39.799016815777414, 47.55396443493713, 55.28191090710056]
-0d10m28.78000259s
-0d12m35.5733052s
-0d14m42.18857103s
-0d16m19.39270021s


In [137]:
import sys
import os
sys.path.append(os.path.abspath("d:\Library\Astronomy\VākyaPañcāṅga\SPA_c_implementation"))
import siddhanta_model

In [164]:
def ayanamsha (jd):
    d = jd - 2415020.0 # AD1900 Jan 1
    p = 50.2564 + 0.000222*d/365.25
    return 22+27/60+37/3600-10/216000 + (p/3600)*(d/365.25)

siddhanta_model.to_dms(ayanamsha(Time('2100-11-20T00:00:00').jd))

[25.0, 16.0, 1.0, 26.0, 46.0]

In [216]:
import numpy as np
from numpy import sin, arcsin, cos, arccos, arctan2, pi

def eq2ec(alpha, delta, epsilon):
    beta = arcsin(delta)*cos(epsilon) - cos(delta)*sin(epsilon)*sin(alpha)
    coslambda = cos(alpha)*cos(delta)/cos(beta)
    sinlambda = (sin(delta)*sin(epsilon)+cos(delta)*cos(epsilon)*sin(alpha))/cos(beta)
    long = arctan2(sinlambda, coslambda)
    if long < 0:
        long = long + 2*pi
    return [Angle(long, unit='rad'), Angle(beta, unit='rad')]
# print([eq2ec(x.icrs.ra.rad, x.icrs.dec.rad, ) for x in ephem2029_coords])

[dms_tuple(d=0.0, m=0.0, s=0.660355525551465), dms_tuple(d=0.0, m=59.0, s=36.03586477526534), dms_tuple(d=1.0, m=59.0, s=9.37545845679935), dms_tuple(d=2.0, m=58.0, s=40.37798584107364), dms_tuple(d=3.0, m=58.0, s=8.822701121001018), dms_tuple(d=4.0, m=57.0, s=35.74707562549008), dms_tuple(d=5.0, m=56.0, s=59.55643914328448), dms_tuple(d=6.0, m=56.0, s=21.29121838536264), dms_tuple(d=7.0, m=55.0, s=40.73603819245335), dms_tuple(d=8.0, m=54.0, s=58.97447326855868), dms_tuple(d=9.0, m=54.0, s=14.379078463950492), dms_tuple(d=10.0, m=53.0, s=28.076979758723724), dms_tuple(d=11.0, m=52.0, s=38.44498158161855), dms_tuple(d=12.0, m=51.0, s=49.367275079670634), dms_tuple(d=13.0, m=50.0, s=56.395400224873526), dms_tuple(d=14.0, m=50.0, s=2.1205921461465493), dms_tuple(d=15.0, m=49.0, s=7.605534145005635), dms_tuple(d=16.0, m=48.0, s=9.826724191217977), dms_tuple(d=17.0, m=47.0, s=11.34593615130683), dms_tuple(d=18.0, m=46.0, s=10.484959288245363), dms_tuple(d=19.0, m=45.0, s=7.022493100870975)

In [226]:
files = ['./Stellarium_ephemeris/ephem2029.csv', './Stellarium_ephemeris/ephem2030.csv']
obliquities = [Angle('23d26m09.4s'), Angle('23d26m06.5s')]
for i in range(len(files)):
    ephem = pd.read_csv(files[i])
    obliquity = obliquities[i]
    ephem_coords = [SkyCoord(ra=Angle(ephem[' RA (J2000)'][i]), dec=Angle(ephem[' Dec (J2000)'][i]), frame='icrs') for i in range(len(ephem[' RA (J2000)']))]
    ephem_ecliptic = [eq2ec(x.icrs.ra.rad, x.icrs.dec.rad, obliquity.rad) for x in ephem_coords]
    # print([e[0].dms for e in ephem_ecliptic])
    print([e.icrs.ra.dms for e in ephem_coords])

[dms_tuple(d=0.0, m=0.0, s=1.4999999999999998), dms_tuple(d=0.0, m=54.0, s=41.99999999999932), dms_tuple(d=1.0, m=49.0, s=20.999999999999233), dms_tuple(d=2.0, m=43.0, s=58.49999999999838), dms_tuple(d=3.0, m=38.0, s=34.49999999999889), dms_tuple(d=4.0, m=33.0, s=10.499999999999403), dms_tuple(d=5.0, m=27.0, s=44.999999999994884), dms_tuple(d=6.0, m=22.0, s=19.49999999999676), dms_tuple(d=7.0, m=16.0, s=53.999999999998636), dms_tuple(d=8.0, m=11.0, s=29.999999999999147), dms_tuple(d=9.0, m=6.0, s=5.999999999993264), dms_tuple(d=10.0, m=0.0, s=43.49999999999241), dms_tuple(d=10.0, m=55.0, s=20.99999999999156), dms_tuple(d=11.0, m=50.0, s=2.9999999999930083), dms_tuple(d=12.0, m=44.0, s=44.99999999999446), dms_tuple(d=13.0, m=39.0, s=29.99999999999318), dms_tuple(d=14.0, m=34.0, s=19.499999999994202), dms_tuple(d=15.0, m=29.0, s=10.499999999987466), dms_tuple(d=16.0, m=24.0, s=5.999999999989427), dms_tuple(d=17.0, m=19.0, s=4.499999999982265), dms_tuple(d=18.0, m=14.0, s=5.99999999999795